In [1]:
import requests
import xarray as xr
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import copernicusmarine
from pprint import pprint
import numpy as np

In [2]:
copernicusmarine.__version__

'1.3.2'

In [3]:
dataset_id = "cmems_mod_glo_phy_my_0.083deg_P1D-m"

In [4]:
catalogue = copernicusmarine.describe(contains=[dataset_id], include_datasets=True)
datasets = catalogue['products'][0]['datasets']

In [5]:
for d in datasets:
    print("Dataset Name:" , d['dataset_name']);     print(" ")
    print("Dataset ID: ", d['dataset_id']);     print(" ")

    for service in d['versions'][0]['parts'][0]['services']:
        print(service['service_type'])
        print(service['service_type']['service_name'])
        variable_list = []
        for variable in service['variables']:
            variable_list.append(variable['short_name'])
        print(variable_list)
        print(" ")

Dataset Name: daily mean fields from Global Ocean Physics Analysis and Forecast updated Daily
 
Dataset ID:  cmems_mod_glo_phy_my_0.083deg_P1D-m
 
{'service_name': 'original-files', 'short_name': 'files'}
original-files
['bottomT', 'mlotst', 'siconc', 'sithick', 'so', 'thetao', 'uo', 'usi', 'vo', 'vsi', 'zos']
 
{'service_name': 'wmts', 'short_name': 'wmts'}
wmts
['bottomT', 'mlotst', 'siconc', 'sithick', 'so', 'thetao', 'uo', 'usi', 'vo', 'vsi', 'zos']
 
{'service_name': 'arco-geo-series', 'short_name': 'geoseries'}
arco-geo-series
['bottomT', 'mlotst', 'siconc', 'sithick', 'so', 'thetao', 'uo', 'usi', 'vo', 'vsi', 'zos']
 
{'service_name': 'arco-time-series', 'short_name': 'timeseries'}
arco-time-series
['bottomT', 'mlotst', 'siconc', 'sithick', 'so', 'thetao', 'uo', 'usi', 'vo', 'vsi', 'zos']
 


In [6]:
#downloading geostrophic velocities from CMEMS Data Store
data_request = {
   "longitude" : [-126, -106], 
   "latitude" : [20, 46],
   "time" : ["1993-01-01", "2016-12-30"],
   "variables" : ["thetao", "so"]
}

In [8]:
copernicusmarine.subset(
    dataset_id = dataset_id,
    dataset_version="202311",
    minimum_longitude = data_request["longitude"][0],
    maximum_longitude = data_request["longitude"][1],
    minimum_latitude = data_request["latitude"][0],
    maximum_latitude = data_request["latitude"][1],
    variables=data_request["variables"], 
    start_datetime = data_request["time"][0],
    end_datetime = data_request["time"][1],
    username = "ameza",
    password = "Ameza1998", 
    minimum_depth=0.49402499198913574,
    maximum_depth=0.49402499198913574,
    output_directory = "", #downloads directly to cwd
    output_filename = "glorys_temp_daily.nc",
    overwrite_output_data = True,
    force_download = False, 
    file_format = "netcdf",

)

INFO - 2024-08-20T04:17:41Z - You forced selection of dataset version "202311"
INFO - 2024-08-20T04:17:41Z - Dataset part was not specified, the first one was selected: "default"
INFO - 2024-08-20T04:17:44Z - Service was not specified, the default one was selected: "arco-time-series"
INFO - 2024-08-20T04:17:47Z - Downloading using service arco-time-series...
INFO - 2024-08-20T04:17:53Z - <xarray.Dataset> Size: 11GB
Dimensions:    (depth: 1, latitude: 313, longitude: 241, time: 8765)
Coordinates:
  * depth      (depth) float32 4B 0.494
  * latitude   (latitude) float32 1kB 20.0 20.08 20.17 ... 45.83 45.92 46.0
  * longitude  (longitude) float32 964B -126.0 -125.9 -125.8 ... -106.1 -106.0
  * time       (time) datetime64[ns] 70kB 1993-01-01 1993-01-02 ... 2016-12-30
Data variables:
    thetao     (time, depth, latitude, longitude) float64 5GB dask.array<chunksize=(2081, 1, 16, 16), meta=np.ndarray>
    so         (time, depth, latitude, longitude) float64 5GB dask.array<chunksize=(2081, 

  Y


INFO - 2024-08-20T04:17:56Z - Writing to local storage. Please wait...


  0%|          | 0/21802 [00:00<?, ?it/s]

INFO - 2024-08-20T04:22:29Z - Successfully downloaded to glorys_temp_daily.nc


PosixPath('glorys_temp_daily.nc')

In [7]:
from dask.distributed import Client
client = Client(n_workers=4, memory_limit='2GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 7.45 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51758,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 7.45 GiB
Comm: tcp://127.0.0.1:51770,Total threads: 3
Dashboard: http://127.0.0.1:51772/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:51761,


In [11]:
glorys = xr.open_mfdataset("glorys_temp_daily.nc", engine = "netcdf4", 
                           chunks = {"latitude":20, "longitude":20, "time":-1})
glorys = glorys.astype("float32")